In [19]:
import pandas as pd
from pandas.io import sql
import numpy as np

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [11]:
airbnb_file="data/singapore-airbnb/listings.csv"
o_df = pd.read_csv(airbnb_file)
o_df = o_df.fillna(0)

In [16]:

def prep_price_preds(o_df):
    higher_b = o_df["price"].quantile(0.99)
    lower_b = o_df["price"].quantile(0.005)

    m_df = o_df[o_df['price'] <=higher_b]
    df = m_df[m_df['price'] >= lower_b].copy()
    df.drop("id", axis=1,inplace=True)
    df.drop("host_name", axis=1,inplace=True)
    df.drop("last_review", axis=1,inplace=True)
    df.drop("name", axis=1,inplace=True)
    
    neighbours = list(df["neighbourhood"].unique())
    neighbours_dict = {}
    for i in range(len(neighbours)):
        neighbours_dict[neighbours[i]] = i
    
    
    room_types = []
    for index, row in df.iterrows():
        if row['room_type'] == "Private room":
            room_types.append(0)
        elif row['room_type'] == "Entire home/apt":
            room_types.append(1)
        else:
            room_types.append(2)
    df["room_type"] = room_types
    
    neighbourhood_groups = []
    for index, row in df.iterrows():
        if row['neighbourhood_group'] == "North Region":
            neighbourhood_groups.append(0)
        elif row['neighbourhood_group'] == "Central Region":
            neighbourhood_groups.append(1)
        elif row['neighbourhood_group'] == "East Region":
            neighbourhood_groups.append(2)
        elif row['neighbourhood_group'] == "West Region":
            neighbourhood_groups.append(3)
        else:
            neighbourhood_groups.append(4)
    df["neighbourhood_group"] = neighbourhood_groups
    
    
    neighbour_types = []
    for index, row in df.iterrows():
            neighbour_types.append(neighbours_dict[row["neighbourhood"]])
    df["neighbourhood"] = neighbour_types
    
    
    for column in df.columns:
        df[column] = pd.to_numeric(df[column])
    return df

In [17]:
df = prep_price_preds(o_df)

In [18]:
df

host_id  neighbourhood_group  neighbourhood  latitude  longitude  \
0        266763                    0              0   1.44255  103.79580   
1        227796                    1              1   1.33235  103.78521   
2        266763                    0              0   1.44246  103.79667   
3        367042                    2              2   1.34541  103.95712   
4        367042                    2              2   1.34567  103.95963   
5        367042                    2              2   1.34702  103.96103   
6        367042                    2              2   1.34348  103.96337   
7       1017645                    2              3   1.32304  103.91363   
8       1017645                    2              3   1.32458  103.91163   
9       1017645                    2              3   1.32461  103.91191   
10      1439258                    1              4   1.28875  103.80814   
11      1439258                    1              4   1.28837  103.81099   
12       367042                    2              2   1.34561  103.95984   
13      1521514                    1              5   1.31125  103.83816   
14      1439258                    1              4   1.28976  103.80896   
15      1439258                    1              4   1.28677  103.81244   
16      1439258                    1              4   1.28537  103.81087   
17       367042                    2              2   1.34943  103.95951   
18      1759905                    1              6   1.31420  103.90232   
19      1521514                    1              5   1.31150  103.83759   
20      1521514                    1              5   1.31159  103.83805   
21      1975201                    1              7   1.29345  103.83562   
22       367042                    2              2   1.34838  103.95920   
23      2386154                    2              2   1.34816  103.93238   
24      1030128                    3              8   1.34135  103.71139   
25      2996022                    1              1   1.33171  103.79383   
26      3212572                    1              9   1.30109  103.85234   
27      1289033                    1              1   1.31016  103.79853   
28      3648224                    1             10   1.30901  103.79639   
29      3824517                    4             11   1.36743  103.87288   
...         ...                  ...            ...       ...        ...   
7876    8948251                    1             19   1.31163  103.86090   
7877    6632570                    0              0   1.44052  103.79784   
7878   32184450                    1              6   1.31468  103.89816   
7879   32184450                    1              6   1.31574  103.89829   
7880  262337792                    1              7   1.29560  103.83803   
7881    7532643                    1              4   1.28474  103.82882   
7882    8948251                    1             19   1.31004  103.86208   
7883  228823978                    1             21   1.27875  103.85505   
7884   43591543                    1              7   1.29820  103.83444   
7885   57336779                    1             23   1.28942  103.84548   
7886  259978610                    2              2   1.35556  103.94428   
7887  153045562                    1             19   1.30329  103.86155   
7888  122219616                    4             25   1.37398  103.83906   
7889    1725521                    1              6   1.31413  103.88857   
7891  288249975                    1             17   1.29481  103.82840   
7892   10823332                    1             21   1.28161  103.85350   
7893  108818897                    1             10   1.28111  103.78636   
7894  108818897                    1             10   1.28129  103.78607   
7895  108818897                    1             10   1.28001  103.78737   
7896    8200399                    1              6   1.31557  103.88979   
7897  262337792                    1              7   1.

In [23]:
def price_prediction(df,queries=[]):
    X = df.drop("price",axis=1).values
    y = df["price"].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)
    
    model = RandomForestRegressor(n_estimators=110, max_depth=10)
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    print(X_test)
    return mean_squared_error(y_test,preds)
    

In [24]:
price_prediction(df)

[[2.60100811e+08 1.00000000e+00 6.00000000e+00 ... 0.00000000e+00
  2.40000000e+01 2.99000000e+02]
 [2.08726450e+07 3.00000000e+00 2.60000000e+01 ... 1.20000000e-01
  2.00000000e+00 3.31000000e+02]
 [2.75025685e+08 1.00000000e+00 2.40000000e+01 ... 1.00000000e+00
  2.00000000e+00 1.72000000e+02]
 ...
 [6.34489120e+07 1.00000000e+00 2.30000000e+01 ... 0.00000000e+00
  4.30000000e+01 2.20000000e+01]
 [1.56564680e+07 1.00000000e+00 2.00000000e+01 ... 1.60000000e-01
  1.00000000e+00 0.00000000e+00]
 [3.24101680e+07 1.00000000e+00 4.00000000e+00 ... 4.45000000e+00
  1.00000000e+00 1.17000000e+02]]


5663.6807038545385